1단계: 파일 로딩 & 기본 세팅

In [39]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import os
import platform


if platform.system() == 'Darwin': # Mac
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)


df =pd.read_csv("../../data/2.processed/processed_final.csv")

df.head()
df.columns

Index(['시점', '행정구역(시)', '행정구역(시군구)별', '총인구수', '남자인구수', '여자인구수', '영아기', '아동기',
       '청소년기', '청년기', '중년기', '노년기', 'CCTV', '1인가구비율', '경찰서수', '강력범죄_살인계',
       '강력범죄_강도', '강력범죄_성범죄', '절도범죄_전체', '폭력범죄_전체', '지능범죄_재산범죄', '풍속범죄_전체',
       '특별경제범죄_전체', '마약범죄_전체', '보건범죄_전체', '환경범죄_전체', '교통범죄_전체', '노동범죄_전체',
       '안보범죄_전체', '선거범죄_전체', '병역범죄_전체', '기타범죄_전체'],
      dtype='object')

2단계: 범죄율, 여성비율, 노년기 비율 생성

In [40]:
# 1) 총범죄율: 모든 범죄 관련 컬럼 합산 (이미 per 1000명 값이니까 합해도 per 1000명 지수)
crime_cols = [col for col in df.columns if "범죄" in col]

print("총범죄율에 포함되는 컬럼들:")
print(crime_cols)

df["총범죄율"] = df[crime_cols].sum(axis=1)

# 2) 여성_비율 (%)
if "여성_비율" not in df.columns:
    df["여성_비율"] = df["여자인구수"] / df["총인구수"] * 100

# 3) 노년기_비율 (%)
if "노년기_비율" not in df.columns:
    df["노년기_비율"] = df["노년기"] / df["총인구수"] * 100

df[["총범죄율", "여성_비율", "노년기_비율"]].head()

총범죄율에 포함되는 컬럼들:
['강력범죄_살인계', '강력범죄_강도', '강력범죄_성범죄', '절도범죄_전체', '폭력범죄_전체', '지능범죄_재산범죄', '풍속범죄_전체', '특별경제범죄_전체', '마약범죄_전체', '보건범죄_전체', '환경범죄_전체', '교통범죄_전체', '노동범죄_전체', '안보범죄_전체', '선거범죄_전체', '병역범죄_전체', '기타범죄_전체']


,총범죄율,여성_비율,노년기_비율
0,25.534301,51.604192,31.155578
1,48.832242,52.195204,25.957243
2,18.491689,51.122960,30.016196
3,23.467339,51.555395,39.577318
4,19.511086,52.062928,30.245025


3단계: 단순회귀 공통 함수 정의 

In [41]:
def run_simple_ols(df, y_col, x_col):
    """
    단순회귀: y_col ~ x_col
    df    : 전체 데이터프레임
    y_col : 종속변수 이름 (문자열)
    x_col : 독립변수 이름 (문자열)
    """
    # 1) 필요한 두 변수만 추출 
    data = df[[y_col, x_col]]

    # 2) y, X 분리
    y = data[y_col]
    X = data[[x_col]]
    
    # 3) 상수항 추가 (절편 β0)
    X = sm.add_constant(X)
    
    # 4) OLS 적합
    model = sm.OLS(y, X).fit()
    
    # 5) 결과 출력
    print(f"\n=== {y_col} ~ {x_col} ===")
    print(model.summary())
    
    return model


4단계: 가설별 단순회귀 모형 검증

In [42]:
# 가설 1: 1인가구 비율이 높을수록 절도범죄율이 높다.
model0 = run_simple_ols(df, "절도범죄_전체", "1인가구비율")



=== 절도범죄_전체 ~ 1인가구비율 ===
                            OLS Regression Results                            
Dep. Variable:                절도범죄_전체   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     16.91
Date:                Fri, 28 Nov 2025   Prob (F-statistic):           5.28e-05
Time:                        23:24:04   Log-Likelihood:                -498.69
No. Observations:                 260   AIC:                             1001.
Df Residuals:                     258   BIC:                             1009.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2724     

1인가구비율이 1%p 증가할 때, 절도범죄율(인구 1000명당)이 평균 약 0.085건 증가하는 양(+)의 관계가 통계적으로 유의하게 나타났다(p < 0.001).  
다만 단일 변수로 설명되는 비율은 약 6.2% 수준으로, 설명력 자체는 크지 않다.

> 결론: 양(+)의 유의한 관계가 관찰되므로, 가설 1은 데이터에 의해 지지된다.

In [43]:
# 가설 2: CCTV가 많은 지역은 범죄율(총범죄율)이 더 낮다.
model1 = run_simple_ols(df, "총범죄율", "CCTV")


=== 총범죄율 ~ CCTV ===
                            OLS Regression Results                            
Dep. Variable:                   총범죄율   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                   0.03497
Date:                Fri, 28 Nov 2025   Prob (F-statistic):              0.852
Time:                        23:24:04   Log-Likelihood:                -1061.0
No. Observations:                 260   AIC:                             2126.
Df Residuals:                     258   BIC:                             2133.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         26.7224      0.95

CCTV 수의 회귀계수는 음(-)의 값을 가지지만, p-value가 0.852로 매우 커서 통계적으로 전혀 유의하지 않다.  
또한 R²가 0에 가깝기 때문에, CCTV 수만으로는 시군구별 총범죄율 변동을 설명하기 어렵다.

> 결론: 단순회귀 기준으로 “CCTV가 많을수록 총범죄율이 낮다”는 근거는 발견되지 않았으며, 가설 2는 기각된다.


In [44]:

# 가설 3: 경찰서가 많은 지역은 범죄율(총범죄율)이 더 낮다.
model2 = run_simple_ols(df, "총범죄율", "경찰서수")



=== 총범죄율 ~ 경찰서수 ===
                            OLS Regression Results                            
Dep. Variable:                   총범죄율   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                  0.002231
Date:                Fri, 28 Nov 2025   Prob (F-statistic):              0.962
Time:                        23:24:04   Log-Likelihood:                -1061.1
No. Observations:                 260   AIC:                             2126.
Df Residuals:                     258   BIC:                             2133.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         26.6782      0.99

경찰서수의 계수는 아주 미세한 음(-)의 값을 보이지만, p-value가 0.962로 유의수준 0.05를 한참 상회한다.  
즉, 경찰서의 개수와 총범죄율 사이에 통계적으로 의미 있는 선형관계는 관찰되지 않았다.

>  결론: “경찰서가 많을수록 총범죄율이 낮다”는 가설 3 역시 단순회귀 결과만으로는 지지되지 않으며, 기각된다.


In [45]:
# 가설 4: 여성 비율이 높은 지역은 성범죄율이 더 높다.
model3 = run_simple_ols(df, "강력범죄_성범죄", "여성_비율")


=== 강력범죄_성범죄 ~ 여성_비율 ===
                            OLS Regression Results                            
Dep. Variable:               강력범죄_성범죄   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     15.63
Date:                Fri, 28 Nov 2025   Prob (F-statistic):           9.93e-05
Time:                        23:24:04   Log-Likelihood:                -1.2705
No. Observations:                 260   AIC:                             6.541
Df Residuals:                     258   BIC:                             13.66
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.7254     

여성 비율이 1%p 증가할 때, 성범죄율(인구 1000명당)이 평균 약 0.0427건 증가하는 양(+)의 관계가 나타났으며, 이 계수는 통계적으로 유의하다(p < 0.001).  
R²는 약 5.7%로, 단일 변수로서의 설명력은 제한적이지만 “여성 비율이 높을수록 성범죄율이 다소 높은 경향”은 데이터에서 확인된다.

>  결론: 가설 4는 방향과 유의성 측면에서 데이터에 의해 지지된다.

In [46]:
# 가설 5: 고령층 비율이 높은 지역은 지능범죄율이 더 높다.
model4 = run_simple_ols(df, "지능범죄_재산범죄", "노년기_비율")


=== 지능범죄_재산범죄 ~ 노년기_비율 ===
                            OLS Regression Results                            
Dep. Variable:              지능범죄_재산범죄   R-squared:                       0.152
Model:                            OLS   Adj. R-squared:                  0.148
Method:                 Least Squares   F-statistic:                     46.10
Date:                Fri, 28 Nov 2025   Prob (F-statistic):           7.69e-11
Time:                        23:24:04   Log-Likelihood:                -631.29
No. Observations:                 260   AIC:                             1267.
Df Residuals:                     258   BIC:                             1274.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.5922   

노년기 비율이 1%p 증가할 때, 지능범죄 재산범죄율(인구 1000명당)은 평균 약 0.074건 감소하는 음(-)의 관계가 통계적으로 유의하게 나타났다(p < 0.001).  
또한 R² = 0.152로, 이전 단순회귀들에 비해 상대적으로 높은 편이며, 고령층 비율이 지능범죄율과 반비례 관계에 있음을 시사한다.

즉, 가설에서 예상한 것(“노년층 비율 ↑ → 지능범죄율 ↑”)과는 반대 방향의 결과가 나왔다.

>  결론: 고령층 비율이 높을수록 지능범죄율은 오히려 낮아지는 경향이 관찰되며, 가설 5는 기각된다.


### 단순회귀 결과 요약

- 가설 1  
  - 1인가구비율이 1%p 증가할 때 절도범죄율은 평균 약 0.085건(인구 1000명당) 증가하는 양(+)의 관계가 나타났으며, 이는 통계적으로 유의하였다(p < 0.001). → 가설 1 지지

- 가설 2  
  - CCTV 수의 계수는 매우 작은 음(-)의 값이었으나 p = 0.852로 유의하지 않았고, R²도 0에 가까웠다. → CCTV 수와 총범죄율 사이의 뚜렷한 관계는 확인되지 않음, 가설 2 기각

- 가설 3  
  - 경찰서수 역시 계수는 음(-)이지만 p = 0.962로 전혀 유의하지 않았고, 설명력도 거의 없었다. → 경찰서 개수와 총범죄율의 선형관계는 유의하지 않으며, 가설 3 기각

- 가설 4  
  - 여성_비율이 1%p 증가할 때 성범죄율은 평균 약 0.043건(인구 1000명당) 증가하는 양(+)의 관계가 나타났고, p < 0.001로 통계적으로 유의하였다. → 가설 4 지지

- 가설 5  
  - 노년기_비율이 1%p 증가할 때 지능범죄_재산범죄율은 평균 약 0.074건 감소하는 음(-)의 관계가 나타났으며(p < 0.001), R² ≈ 0.152로 다른 단순회귀들보다 설명력이 높은 편이었다.  
  - 즉, 가설에서 예상한 것(“노년층 비율 ↑ → 지능범죄율 ↑”)과는 반대로, 노년층 비율이 높을수록 지능범죄율은 낮아지는 경향을 보여 가설 5는 기각된다.

→ 정리하면, 일부 인구 구조 변수(1인가구비율, 여성_비율, 노년기_비율)는 특정 범죄 유형과 유의한 관련성을 보였으나,  
CCTV 수나 경찰서 수만으로는 총범죄 수준을 설명하기 어려운 것으로 나타났다.
